In [1]:
# fast trigonometry

# 1. Arctan

In [2]:
import numpy as np

In [29]:
 
def cubic(x):
    m = x.shape[0]
    X = np.empty((m, 3))
    X[:,0] = x ; X[:,1] = x * x; X[:,2] = x * x * x
    return X

def quad(x):
    m = x.shape[0]
    X = np.empty((m, 2))
    X[:,0] = x ; X[:,1] = x * x;
    return X

def linear(x):
    X = np.expand_dims(x,1)
    return X

In [ ]:
from sklearn.linear_model import LinearRegression
# 1. a + bx + cx^2 + dx^3
_size = 1000000
x = np.clip(np.abs(np.random.randn(_size)) * np.pi, 0, np.pi)
gt = np.arctan(x)

In [32]:
X = cubic(x)
model = LinearRegression()
model.fit(X, gt)
print(model.coef_)
print(model.intercept_)

[ 1.10382257 -0.36105955  0.04400221]
-0.00492334997743038


In [41]:
pred = model.predict(X)
res = (pred - gt)
idx = np.argmax(res)
print("MAX err", "idx", idx, "value", res[idx], "rad", X[idx,0])
predict = cubic(np.array([[2]]))
print(model.predict(predict), " vs ", np.arctan(predict))


MAX err idx 141589 value 0.005227865735183412 rad 1.7133812981746157
[1.11050127]  vs  [[1.10714872 1.32581766 1.44644133]]


In [34]:
lsize = 5000
xl = np.linspace(5,500,lsize)
gtxl = np.arctan(xl)

In [35]:
Xl = cubic(xl) #quad(xl)
modell = LinearRegression()
modell.fit(Xl, gtxl)
print(modell.coef_)
print(modell.intercept_)

[ 7.65325825e-04 -2.76103373e-06  3.04822121e-09]
1.50443794904616


In [36]:
pred = modell.predict(Xl)
res = (pred - gtxl)
idx = np.argmax(res)
print("MAX err", "idx", idx, "value", res[idx], "rad", Xl[idx,0])
predict = cubic(np.array([[5]]))
print(modell.predict(predict), " vs ", np.arctan(predict))

MAX err idx 0 value 0.13479516641067524 rad 5.0
[1.50819593]  vs  [[1.37340077 1.53081764 1.5627965 ]]


In [44]:
val = 6
print(np.arctan(val))
print(np.pi / 2 - 1 / val)

1.4056476493802699
1.4041296601282298


In [78]:
def fast_arctan(x):
    a0 = 1.10382257
    a1 = -0.36105955
    a2 = 0.04400221
    b = -0.00492334997743038
    sign = 1 if x >= 0 else -1
    if (abs(x) > np.pi):
        # https://math.stackexchange.com/questions/982838/asymptotic-approximation-of-the-arctangent/982859
        return (sign*(np.pi / 2) - 1/x)
    elif abs(x) < 5e-2:
        # for small x
        return x
    else:
        # by regression
        ax = abs(x)
        x2 = ax * ax
        x3 = x2 * ax
        return sign * (b + a0 * ax + a1 * x2 + a2 * x3)

In [85]:
val = -0.049
print(np.sin(val), "vs", fast_arctan(val))

-0.04898039418715918 vs -0.049


In [86]:
np.pi

3.141592653589793